# Bayesian Linear Regression Construction with Pyro

One of the most beneficial uses for probabilistic programming is the construction of probabilistic machine learning models.

Generally speaking, machine learning models are designed as functions, or mappings, from an input to an output. We have some input $X$ which is an $m\times n$ matrix of $m$ data points with $n$ features each, and we want to construct some mapping $f$ which transforms the features into predictions of some unknown values $\vec{y} = \mathbb{R}^n$

in essence, we're interested in a function:

$$f(X) = y$$

and we'd like our ML models to approximate it as best as possible, ideally while also giving us an idea for **why** the produced certain predictions.

One of the basic ML models which accomplishes both is the linear regression which simply maps $y$ as a linear response of $X$ given some coefficients, $\beta$, such that

$$ y = \beta X$$

However, one major drawback of simple linear regressions is that they create point estimates of $\beta$ without taking uncertainty into account. In this article we'll discuss how to leverage PyTorch and Pyro to produce linear regression models which create uncertainty estimates both for the parameters, as well as for predictions.

## Employing Scikit-learn's Linear Regression

We'll start by exploring a simple linear regression from `sklearn`, and see how it behaves on one of the built in datasets in `sklearn`, the California Housing dataset.

We'll start by importing all our required libraries

In [ ]:
import numpy as np
import time
np.set_printoptions(suppress=True)


import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

Now let's import the housing dataset, and explore its features

In [ ]:
super_start = time.time()
super_results = pd.DataFrame({'LR' :[], 
                 'MCMC unscaled normal': [], 
                 'MCMC scaled normal': [], 
                 'MCMC gamma': [], 
                 'MCMC gamma+cen': [], 
                 'SVI gamma+cen': []})


from sklearn.datasets import fetch_california_housing

california = fetch_california_housing()

X = california.data
y = california.target * 100000

print(f'Data shape is {X.shape}')
print(f'Target shape is {y.shape}')

In [ ]:
plt.figure(figsize=(15,7))
plt.hist(y, bins=100, color="blue", edgecolor="black")
plt.xlabel("Median House Value (in $100K)", size=16)
plt.ylabel("Frequency", size=16)
plt.title("A Histogram of California Median House Values Circa 1990", size=22)
plt.xticks(size=16)
plt.yticks(size=16)
plt.savefig("HouseValues.png", dpi=36*4)
plt.show()



There are two important observations to be made from this data; First, the distribution of prices is not normal, it is closer to a Gamma distribution. Second, there is a very sharp spike of instances at the \\$500,000 mark, this can an indication of "censored" data. In essence, the data might have been recorded up to \\$500,000, at which point it was recorded as "\\$500,000+"

Let's explore how well a linear regression performs on this data:

In [ ]:
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


In [ ]:
max_threshold = 5.0 * 100000

my_linear_regression = LinearRegression().fit(X_train,y_train)

print(my_linear_regression.intercept_)
print(my_linear_regression.coef_)

y_pred = my_linear_regression.predict(X_test)

super_results.loc['runtime', 'LR'] = 0.0

y_small = y_test[y_test<max_threshold]
y_pred_small = y_pred[y_test<max_threshold]

y_pred = np.where(y_pred > max_threshold, max_threshold, y_pred)

In [ ]:
plt.figure(figsize=(15, 5))
plt.scatter(y_pred, y_test, c="blue", edgecolor="black", label=f'R^2={round(r2_score(y_test, y_pred),2)}')
plt.title("Predicted House Price vs Actual House Price", size=22)
plt.xlabel("Predicted House Price (in $100K)", size=16)
plt.ylabel("True House Price (in $100K)", size=16)
plt.ticklabel_format(style='plain')
plt.legend()
plt.show()

In [ ]:
super_results.loc['R^2 censored', 'LR'] = r2_score(y_test, y_pred)
r2_score(y_test, y_pred)

In [ ]:
plt.figure(figsize=(15, 5))
plt.scatter(y_pred_small, y_small, c="blue", edgecolor="black", label=f'R^2={round(r2_score(y_small, y_pred_small),2)}')
plt.title("Predicted House Price vs Actual House Price", size=22)
plt.xlabel("Predicted House Price (in $100K)", size=16)
plt.ylabel("True House Price (in $100K)", size=16)
plt.ticklabel_format(style='plain')
plt.legend()
plt.show()

In [ ]:
super_results.loc['R^2 small', 'LR'] = r2_score(y_small, y_pred_small)
r2_score(y_small, y_pred_small)

While this approach can produce satisfactory results, it suffers from a few main drawbacks; 

* First, the linear model generally ignore the fact that the prices come from a Gamma distribution. Its calculations of the expected value for every point are predicated on the mean coming from a Normal distribution. 
* Second, and more importantly, the simple linear model takes the censored data at face value. That is, it believes the prices of all houses registered as "+\\$500,000" to be \\$500,000. This might cause it to underestimate the effect some features have on house price.
* Third, for each coefficient, we only get a point estimate of its most likely value (under certain incorrect assumptions). However, we might be interested in a range which accounts for uncertainty. For example, we might want to know what is the range of price increases we can expect for each additional bedroom.

To address these problems, we can employ Pyro and PyTorch to construct our own linear model which will address all the pain points just mentioned. 

## Step I - Reconstructing the Linear Model with Pyro

First, let's try and replicate the findings of the simple linear regression with Pyro. This will give us an intuition for how the different Pyro primitives work

In [ ]:
# import all required Pyro/Pytorch libraries
import torch
import pyro
import pyro.distributions as dist

# We will use Markov Chain Monte Carlo (MCMC) methods here, specifically the No U-Turn Sampler (NUTS)
from pyro.infer import MCMC, NUTS

# Fix the random seeds
#torch.manual_seed(0)
#pyro.set_rng_seed(0)

# We'll be timing our execution as well
import time

First, we will define our model in Pyro. Pyro models are defined as functions (actually they are defined as callables, but the simplest callable is a function). The function will accept our features $X$, our target $y$, and also the feature names for easier naming of priors.

In [ ]:
def model_normal(X, y, column_names):
    
    # Define our intercept value
    linear_combination = pyro.sample(f"beta_intercept", dist.Normal(0.0, 1.0))
    
    
    betas = pyro.sample(f"beta_coefficients", dist.Normal(0.0, 1.0).expand([X.shape[1]]).to_event(1)).double()
    
    linear_combination = linear_combination + torch.matmul(X, betas)
    
    # Define a sigma value for the random error
    sigma = pyro.sample("sigma", dist.HalfCauchy(scale=10.0))
    
    # For a simple linear model, the expected mean is the linear combination of parameters
    mean = linear_combination
    
    
    with pyro.plate("data", y.shape[0]):
        
        # Assume our expected mean comes from a normal distribution
        outcome_dist = dist.Normal(mean, sigma)
        
        # Condition the expected mean on the observed target y
        observation = pyro.sample("obs", outcome_dist, obs=y)

In [ ]:
def model_normal_old(X, y, column_names):
    
    # Define our intercept value
    linear_combination = pyro.sample(f"beta_intercept", dist.Normal(0.0, 1.0))
    
    
    
    # Define each coefficient using the column name
    for i in range(0, X.shape[1]):
        beta_i = pyro.sample(f"beta_{column_names[i]}", dist.Normal(0.0, 1.0))
        linear_combination = linear_combination + (beta_i * X[:, i])
    
    # Define a sigma value for the random error
    sigma = pyro.sample("sigma", dist.HalfCauchy(scale=10.0))
    
    # For a simple linear model, the expected mean is the linear combination of parameters
    mean = linear_combination
    
    
    with pyro.plate("data", y.shape[0]):
        
        # Assume our expected mean comes from a normal distribution
        outcome_dist = dist.Normal(mean, sigma)
        
        # Condition the expected mean on the observed target y
        observation = pyro.sample("obs", outcome_dist, obs=y)

In essence what we've done here is define our linear regression as the following linear combination of parameters

$$y = \beta_0 + \beta_1x_1 + \beta_2x_2 + ... + \beta_nx_n + \mathcal{N}(0, \sigma^2)$$

However, unlike traditional linear regressions, we've defined each beta coefficient to be a distribution instead of a single values. That is, for each coefficient, we can ask what is the range of possible values this coefficient can assume given the data we observed. We gave a name to each of those distributions (e.g. "`beta_intercept`") for easy reference later.

We had to define priors on each coefficient. A prior is like our "best guess" for that coefficient. Once the priors are defined, we can ask Pyro to update them into better and better guesses through the magic of MCMC samplers

In [ ]:
SAMPLE_NUMBER = 3000


# Turn out numpy data into PyTorch 
# tensors
X_train_torch = torch.tensor(X_train, dtype=torch.float64)
y_train_torch = torch.tensor(y_train, dtype=torch.float64)



# Clear the parameter storage
pyro.clear_param_store()

# Initialize our No U-Turn Sampler
my_kernel = NUTS(model_normal, max_tree_depth=7)

# Employ the sampler in an MCMC sampling 
# algorithm, and sample 3100 samples. 
# Then remove the first 100
my_mcmc1 = MCMC(my_kernel,
                num_samples=SAMPLE_NUMBER,
                warmup_steps=100)


# Let's time our execution as well
start_time = time.time()

# Run the sampler
my_mcmc1.run(X_train_torch, 
             y_train_torch, 
             california.feature_names)

end_time = time.time()

print(f'Inference ran for {round(end_time -  start_time, 2)} seconds')

super_results.loc['runtime', 'MCMC unscaled normal'] = round(end_time -  start_time, 2)



I am not going to explain in detail how the sampling works here, but if you are interested in a breakdown of what has happened here, I recommend that you check out [my previous post](https://towardsdatascience.com/probabilistic-programming-with-pyro-and-kitchen-scale-f8d6a5d9ae0f) which explores the use of MCMC methods to optimize one parameter.

Once our sampler is finished, we can look at the summary data for each beta value.

In [ ]:
my_mcmc1.summary()

Let's grab the individual samples from our sampler, let's turn those into a dataframe (they are returned as a dictionary).
We can grab the mean of each distribution as a coefficient point estimate, and then calculate a set of predictions for our data points. Then we can compare them to our known values for house prices

In [ ]:
beta_df = pd.DataFrame(my_mcmc1.get_samples())

In [ ]:
beta_df = pd.DataFrame(my_mcmc1.get_samples())
beta_df['beta_coefficients'] = beta_df['beta_coefficients'].apply(lambda x: [i.detach().numpy() for i in x])


df_columns = [f'beta_{i}' for i in california.feature_names]
coefficient_df = pd.DataFrame(beta_df['beta_coefficients'].tolist(), 
                              index=beta_df['beta_coefficients'].index, columns=df_columns)
beta_df = pd.concat([beta_df, coefficient_df], axis=1).drop('beta_coefficients', axis=1)



extra_col_name = beta_df.columns[1]
extra_col_values = beta_df[extra_col_name]
beta_df.drop(extra_col_name, axis=1, inplace=True)
beta_df[extra_col_name] = extra_col_values
beta_df

In [ ]:
def predict_linear_combination(beta_df, X):
    
    # Don't grab the last column, that is our estimate of the error standard deviation, "sigma"
    coefficients = beta_df.iloc[:, :-1].mean()
    coefficients

    # Find our linear combination again
    linear_combination = X.dot(coefficients[1:]) + coefficients.iloc[0]
    
    return linear_combination

In [ ]:
linear_combination = predict_linear_combination(beta_df, X_test)

In [ ]:
# Our predictions are the linear combination
y_pred = linear_combination

# Find all the predictions for houses below $500,000 in value
y_pred_small = y_pred[y_test<max_threshold]

# Censor our predictions as well
y_pred= np.where(y_pred > max_threshold, max_threshold, y_pred)

We can compare our entire dataset. As well as our estimate of non-censored data.

In [ ]:
plt.figure(figsize=(15, 5))
plt.scatter(y_pred, y_test, c="blue", edgecolor="black", label=f'R^2={round(r2_score(y_test, y_pred), 2)}')
plt.title("Predicted House Price vs Actual House Price (Entire Dataset)", size=22)
plt.xlabel("Predicted House Price (in $100K)", size=16)
plt.ylabel("True House Price (in $100K)", size=16)
plt.ticklabel_format(style='plain')
plt.legend()
plt.show()

In [ ]:
super_results.loc['R^2 censored', 'MCMC unscaled normal'] = r2_score(y_test, y_pred)
print(r2_score(y_test, y_pred))

In [ ]:
plt.figure(figsize=(15, 5))
plt.scatter(y_pred_small, y_small, c="blue", edgecolor="black", label=f'R^2={round(r2_score(y_small, y_pred_small),2)}')
plt.title("Predicted House Price vs Actual House Price (Non-censored Data Points)", size=22)
plt.xlabel("Predicted House Price (in $100K)", size=16)
plt.ylabel("True House Price (in $100K)", size=16)
plt.ticklabel_format(style='plain')
plt.legend()
plt.show()

In [ ]:
super_results.loc['R^2 small', 'MCMC unscaled normal'] = r2_score(y_small, y_pred_small)
print(r2_score(y_small, y_pred_small))

Well that looks like a disaster! What happened?

Let's define a function that will draw the coefficients' distributions for us when given a coefficient dataframe

In [ ]:
def draw_coefficients(beta_df):
    figure, axis = plt.subplots(5,2, figsize=(15,15))


    for key, ax in zip(beta_df.columns, axis.ravel()):
        ax.set_title(key)
        ax.hist(beta_df[key], color="blue", edgecolor="black", bins=100)

    plt.subplots_adjust(hspace=0.5)

In [ ]:
draw_coefficients(beta_df)

These look interesting... they don't look anything like the coefficients the linear regression from `sklearn` has found. 

Turns out MCMC methods have a hard time with different scales for our data. They work much better when our features and target are scaled. Let's explore the performance of the same methodology with the data scaled

## Scaled data

We are going to scale our X and y data using a standard scaler from `sklearn`. The standard scaler will scale each feature in X in such a way that it has a mean of 0.0, and a standard deviation of 1.0.

This means, instead of trying to find a set of coefficients on the original data


$$y = \beta_0 + \beta_1x_1 + \beta_2x_2 + ... + \beta_nx_n $$

we will find a set of coefficients on the scaled data:

$$y' = \beta'_0 + \beta'_1\frac{x_1 - \mu_1}{\sigma_1} + \beta'_2\frac{x_2 - \mu_2}{\sigma_2} + ... + \beta'_n\frac{x_n - \mu_n}{\sigma_n} $$

Where 

$$y' = \frac{y}{max(y)}$$

That is y is scaled to be between 0-1 and the columns of X have been standardized.

In [ ]:
def model_normal(X, y, column_names):
    
    # Define our intercept value
    linear_combination = pyro.sample(f"beta_intercept", dist.Normal(0.0, 1.0))
    

    betas = pyro.sample(f"beta_coefficients", dist.Normal(0.0, 1.0).expand([X.shape[1]]).to_event(1)).double()
    linear_combination = linear_combination + torch.matmul(X, betas)
    
    # Define a sigma value for the random error
    sigma = pyro.sample("sigma", dist.HalfCauchy(scale=10.0))
    
    # For a simple linear model, the expected mean is the linear combination of parameters
    mean = linear_combination
    
    
    with pyro.plate("data", y.shape[0]):
        
        # Assume our expected mean comes from a normal distribution
        outcome_dist = dist.Normal(mean, sigma)
        
        # Condition the expected mean on the observed target y
        observation = pyro.sample("obs", outcome_dist, obs=y)

In [ ]:
from sklearn.preprocessing import StandardScaler

my_x_scaler = StandardScaler()
X_train_scaled = my_x_scaler.fit_transform(X_train)
X_test_scaled = my_x_scaler.transform(X_test)

y_max = y_train.max()
y_train_scaled = y_train/y_max
y_test_scaled = y_test/y_max

In [ ]:
X_train_torch = torch.tensor(X_train_scaled, dtype=torch.float64)
y_train_torch = torch.tensor(y_train_scaled, dtype=torch.float64)


pyro.clear_param_store()
my_kernel = NUTS(model_normal, max_tree_depth=7)


my_mcmc2 = MCMC(my_kernel,
                num_samples=SAMPLE_NUMBER,
                warmup_steps=100)


start_time = time.time()
my_mcmc2.run(X_train_torch,
             y_train_torch, 
             california.feature_names)
end_time = time.time()

print(f'Inference ran for {round(end_time -  start_time, 2)} seconds')
super_results.loc['runtime', 'MCMC scaled normal'] = round(end_time -  start_time, 2)



my_mcmc2.summary()

Our algorithm ran much faster now, but if we recover the coefficients the algorithm found, those will be the coefficients on the scaled data. We would like to translate them back into the unscaled data so we can ask questions such as: "For each extra bedroom in the house, what will be the effect on the price?"


Luckily, we can manipulate our equation to retrieve the coefficients on the unscaled data. We begin with our original equation

$$y' = \beta'_0 + \beta'_1\frac{x_1 - \mu_1}{\sigma_1} + \beta'_2\frac{x_2 - \mu_2}{\sigma_2} + ... + \beta'_n\frac{x_n - \mu_n}{\sigma_n} $$

and we expand each fraction:

$$y' = \beta'_0 + \beta'_1\frac{x_1}{\sigma_1} - \beta'_1\frac{\mu_1}{\sigma_1} + \beta'_2\frac{x_2}{\sigma_2} - \beta'_2\frac{\mu_2}{\sigma_2} + ... + \beta'_n\frac{x_n}{\sigma_n} - \beta'_n\frac{\mu_n}{\sigma_n} $$

We can then rearrange the equation as follows:

$$y' = \beta'_0 - \beta'_1\frac{\mu_1}{\sigma_1} - \beta'_2\frac{\mu_2}{\sigma_2} ... - \beta'_n\frac{\mu_n}{\sigma_n} + \frac{\beta'_1}{\sigma_1}x_1 + \frac{\beta'_2}{\sigma_2}x_2 + ... + \frac{\beta'_n}{\sigma_n}x_n  $$

Recalling that 

$$y' = \frac{y}{max(y)}$$

we can finally rewrite our formula as follows:

$$y = max(y)\big( \beta'_0 - \sum_{i=1}^n\beta'_i\frac{\mu_i}{\sigma_i} \big) + \frac{\beta'_1 max(y)}{\sigma_1}x_1 + \frac{\beta'_2 max(y)}{\sigma_2}x_2 + ... + \frac{\beta'_n max(y)}{\sigma_n}x_n  $$

We can create a function to perform the processing of the coefficients from the scaled data to the unscaled data

In [ ]:
def create_beta_df(beta_df, x_scaler, feature_names):        
        
    beta_df['beta_coefficients'] = beta_df['beta_coefficients'].apply(lambda x: [i.detach().numpy() for i in x])


    df_columns = [f'beta_{i}' for i in feature_names]
    coefficient_df = pd.DataFrame(beta_df['beta_coefficients'].tolist(), 
                                  index=beta_df['beta_coefficients'].index, columns=df_columns)
    beta_df = pd.concat([beta_df, coefficient_df], axis=1).drop('beta_coefficients', axis=1)



    extra_col_name = beta_df.columns[1]
    extra_col_values = beta_df[extra_col_name]
    beta_df.drop(extra_col_name, axis=1, inplace=True)
    beta_df[extra_col_name] = extra_col_values
    beta_df    
        
    
    i = 0
    for col in beta_df:
        if (col != 'beta_intercept'):
            if ('beta_' in col):

                beta_df['beta_intercept'] -= (beta_df[col] * x_scaler.mean_[i])/x_scaler.scale_[i]
                
                
                beta_df[col] /= x_scaler.scale_[i]

                i += 1

    return beta_df

In [ ]:
beta2_df = pd.DataFrame(my_mcmc2.get_samples())
beta2_df = create_beta_df(beta2_df, my_x_scaler, california.feature_names)
beta2_df *= y_max   # multiply everything by y_max outside the function for reasons that will
                    # become clear later

These distributions seem much closer to the values found by the linear regression from `sklearn`. Let's compare the prediction results

In [ ]:
linear_combination = predict_linear_combination(beta2_df, X_test)

In [ ]:
y_pred = linear_combination
y_pred_small = y_pred[y_test<max_threshold]
y_pred= np.where(y_pred > max_threshold, max_threshold, y_pred)

Again we will compare the results on the entire dataset, as well as on the uncensored labels only

In [ ]:
plt.figure(figsize=(15, 5))
plt.scatter(y_pred, y_test, c="blue", edgecolor="black", label=f'R^2={round(r2_score(y_test, y_pred), 2)}')
plt.title("Predicted House Price vs Actual House Price (Entire Dataset)", size=22)
plt.xlabel("Predicted House Price (in $100K)", size=16)
plt.ylabel("True House Price (in $100K)", size=16)
plt.ticklabel_format(style='plain')
plt.legend()
plt.show()

In [ ]:
super_results.loc['R^2 censored', 'MCMC scaled normal'] = r2_score(y_test, y_pred)
print(r2_score(y_test, y_pred))

In [ ]:
plt.figure(figsize=(15, 5))
plt.scatter(y_pred_small, y_small, c="blue", edgecolor="black", label=f'R^2={round(r2_score(y_small, y_pred_small),2)}')
plt.title("Predicted House Price vs Actual House Price (Non-censored Data Points)", size=22)
plt.xlabel("Predicted House Price (in $100K)", size=16)
plt.ylabel("True House Price (in $100K)", size=16)
plt.ticklabel_format(style='plain')
plt.legend()
plt.show()

In [ ]:
super_results.loc['R^2 small', 'MCMC scaled normal'] = r2_score(y_small, y_pred_small)
print(r2_score(y_small, y_pred_small))

In [ ]:
draw_coefficients(beta2_df)

It seems like we got a comparable performance.

However, we can actually employ Pyro to do better! Recall that we saw our house prices are not normally distributed but in fact follow a Gamma distribution. We can modify our code to reflect that in the model.

## Gamma Distribution

In order to better reflect the house distribution, we can employ a Gamma distribution for our target values. Unlike the Normal distribution which is defined by its mean and standard deviation, the Gamma distribution is defined by two positive parameters which are the shape and the rate.

When constructing our model for a distribution other than normal, we need to employ a **link function** which will translate the linear combination of our parameters to the expected value, or the mean, of the distribution. We also would like to know the relationship between the mean and the distribution parameters. Luckily, for the Gamma distribution this is predefined as:

$$\mu = \frac{shape}{rate}$$

However, if both the shape and rate parameters are positive, that means the mean must be positive as well. We need to make sure that our link function captures that. Therefore, I will use the following link function for the linear equation:

$$ln(\mu) = ln(y') = \beta'_0 + \beta'_1\frac{x_1 - \mu_1}{\sigma_1} + \beta'_2\frac{x_2 - \mu_2}{\sigma_2} + ... + \beta'_n\frac{x_n - \mu_n}{\sigma_n}$$

or 

$$y' = e^{\beta'_0 + \beta'_1\frac{x_1 - \mu_1}{\sigma_1} + \beta'_2\frac{x_2 - \mu_2}{\sigma_2} + ... + \beta'_n\frac{x_n - \mu_n}{\sigma_n}}$$.


Interestingly enough, to recover the coefficients for the unscaled data, the math works out fairly similarly except for the constant. Keeping in mind that:

$$y' = \frac{y}{max(y)}$$

and that

$$ln(\frac{y}{max(y)}) = ln(y) - ln(max(y))$$

We can find that our equation can be written as:

$$ln(y) = \big(ln(max(y)) + \beta'_0 - \sum_{i=1}^n\beta'_i\frac{\mu_i}{\sigma_i} \big) + \frac{\beta'_1}{\sigma_1}x_1 + \frac{\beta'_2}{\sigma_2}x_2 + ... + \frac{\beta'_n}{\sigma_n}x_n  $$

or 

$$y = e^{\big(ln(max(y)) + \beta'_0 - \sum_{i=1}^n\beta'_i\frac{\mu_i}{\sigma_i} \big) + \frac{\beta'_1}{\sigma_1}x_1 + \frac{\beta'_2}{\sigma_2}x_2 + ... + \frac{\beta'_n}{\sigma_n}x_n}$$

In [ ]:
def model_gamma(X, y, column_names):
    
    # We still need to calculate our linear combination
    linear_combination = pyro.sample(f"beta_intercept", dist.Normal(0.0, 1.0))
    
    
    betas = pyro.sample(f"beta_coefficients", dist.Normal(0.0, 1.0).expand([X.shape[1]]).to_event(1)).double()
    linear_combination = linear_combination + torch.matmul(X, betas)
    
    # But now our mean will be e^{linear combination}
    mean = torch.exp(linear_combination)
    
    # We will also define a rate parameter
    rate = pyro.sample("rate", dist.HalfCauchy(scale=10.0))
    
    # Since mean = shape/rate, then the shape = mean * rate
    shape = mean * rate
    
    
    
    # Now that we have the shape and rate parameters for the
    # Gamma distribution, we can draw samples from it and condition
    # them on our observations
    with pyro.plate("data", y.shape[0]):
        
        outcome_dist = dist.Gamma(shape, rate)
        
        observation = pyro.sample("obs", outcome_dist, obs=y)

You'll notice our code is slightly different but we are still calculating a linear combination of our X data and our coefficients, except now we take the `exp` of that combination to get the expected value of our data point. 
We also sample a rate parameter, and use the mean and rate to calculate the appropriate shape parameter.

Given our shape and rate parameters, we can define a Gamma distribution and ask Pyro to optimize the values this distribution depends on (namely, our coefficients and rate parameters) in order to build a model most likely based on our data.

Let's optimize this new model and look at the results

In [ ]:
from sklearn.preprocessing import StandardScaler

my_x_scaler = StandardScaler()

X_train_scaled = my_x_scaler.fit_transform(X_train)
X_test_scaled = my_x_scaler.transform(X_test)

y_max = y_train.max()
y_train_scaled = y_train/y_max
y_test_scaled = y_test/y_max

X_train_torch = torch.tensor(X_train_scaled, dtype=torch.float64)
y_train_torch = torch.tensor(y_train_scaled, dtype=torch.float64)


pyro.clear_param_store()
my_kernel = NUTS(model_gamma, max_tree_depth=7)


my_mcmc3 = MCMC(my_kernel,
                num_samples=SAMPLE_NUMBER,
                warmup_steps=100)



start_time = time.time()
my_mcmc3.run(X_train_torch,
             y_train_torch, 
             california.feature_names)
end_time = time.time()

print(f'Inference ran for {round(end_time -  start_time, 2)} seconds')
super_results.loc['runtime', 'MCMC gamma'] = round(end_time -  start_time, 2)



my_mcmc3.summary()

In [ ]:
beta3_df = pd.DataFrame(my_mcmc3.get_samples())
beta3_df = create_beta_df(beta3_df, my_x_scaler, california.feature_names)

# Notice now we have to use the y_max value slightly differently
beta3_df['beta_intercept'] += np.log(y_max)

Let's compare this models performance by calculating the predictions and comparing them to the observed values

In [ ]:
linear_combination = predict_linear_combination(beta3_df, X_test)

In [ ]:
# We have to take the EXP of the linear combination now to get the expected values
y_pred = np.exp(linear_combination)

# Again, compare for both the censored and uncensored datasets
y_pred_small = y_pred[y_test<max_threshold]
y_pred= np.where(y_pred > max_threshold, max_threshold, y_pred)

In [ ]:
plt.figure(figsize=(15, 5))
plt.scatter(y_pred, y_test, c="blue", edgecolor="black", label=f'R^2={round(r2_score(y_test, y_pred), 2)}')
plt.title("Predicted House Price vs Actual House Price", size=22)
plt.xlabel("Predicted House Price (in $100K)", size=16)
plt.ylabel("True House Price (in $100K)", size=16)
plt.legend()
plt.show()

In [ ]:
super_results.loc['R^2 censored', 'MCMC gamma'] = r2_score(y_test, y_pred)
print(r2_score(y_test, y_pred))

In [ ]:
plt.figure(figsize=(15, 5))
plt.scatter(y_pred_small, y_small, c="blue", edgecolor="black", label=f'R^2={round(r2_score(y_small, y_pred_small),2)}')
plt.title("Predicted House Price vs Actual House Price", size=22)
plt.xlabel("Predicted House Price (in $100K)", size=16)
plt.ylabel("True House Price (in $100K)", size=16)
plt.legend()
plt.show()

In [ ]:
super_results.loc['R^2 small', 'MCMC gamma'] = r2_score(y_small, y_pred_small)
print(r2_score(y_small, y_pred_small))

In [ ]:
draw_coefficients(beta3_df)

We can notice a few interesting differences between the qualitative conclusions this model gives us as opposed to the Normal distribution model:
* First, under the Normal distribution assumption, it seemed that the house price decreases for each additional room, but under the Gamma distribution assumption, the price seems to increase.
* Second, under the Normal distribution assumption, the price seems to increase for each additional bedroom of the house, but under the Gamma distribution assumption, that conclusion does not seem as certain anymore. 

Other parameters seem to qualitatively agree between the two models. However, we want to be certain we correctly identified the quantitative values of the parameters, and we need to account for the censored data to do that.

## Gamma + Censored data

For any given house, we assume that its price comes from a Gamma distribution. However, we've observed in our data that if a house price is above \\$500,000, that price will be censored. The question becomes, how do we quantify this in our model.

We can ask ourselves, "For a given house, what is the probability that its price will be censored?"

The answer is that the probability is the same as the probability that the house price will be above \\$500,000. Well we have a clear way to ask a distribution for the probability a value will be below some threshold, it is the Cumulative Distribution Function (CDF) at that threshold. So 1 - CDF(500,000) will give us the probability that a house value will be above \\$500,000.

We will use that information and add that to the model. We'll say that if the model sees a value below \\$500,000, that is the observed value of the house. But if the model sees a value of \\$500,000, the probability of observing that censoring is the result of a Bernoulli trial, with the probability of censoring "success" being 1-CDF(500,000).

In [ ]:
from scipy.stats import gamma as scipygamma
def model_gamma_cen(X, y, column_names, censored_label):
    pyro.enable_validation(True) 
    
    # Again find the linear combination
    linear_combination = pyro.sample(f"beta_intercept", dist.Normal(0.0, 1.0))
    
    
    #for i in range(0, X.shape[1]):
    #    beta_i = pyro.sample(f"beta_{column_names[i]}", dist.Normal(0.0, 1.0))
    #    linear_combination = linear_combination + (beta_i * X[:, i])
    betas = pyro.sample(f"beta_coefficients", dist.Normal(0.0, 1.0).expand([X.shape[1]]).to_event(1)).double()
    linear_combination = linear_combination + torch.matmul(X, betas)
    
    # The mean is e^{linear combination}
    mean = torch.exp(linear_combination)
    mean = torch.max(mean, torch.tensor(torch.finfo(mean.dtype).eps).double()) # sometimes the linear
                                                                         # combination is too small so
                                                                         # e^ of that becomes 0
                                                                         # We replace it with the min value
                                                                         # torch can have in float32 mode
    
    # Sample a rate as well
    rate = pyro.sample("rate", dist.HalfCauchy(scale=10.0)).clamp(min=torch.finfo(X.dtype).eps)
    #rate = torch.max(rate, torch.tensor(torch.finfo(rate.dtype).eps).double())
    
    # Calculate the shape based on the mean and rate
    shape = mean * rate
    
    
    
    # The data is now divided into two outcomes: censored, and non-censored
    with pyro.plate("data", y.shape[0]):
        
        
        #print(mean.min(), mean.max())
        #print(linear_combination.min())
        
        #print(rate.min(), rate.max())
        #print(shape.min(), shape.max())
        
        outcome_dist = dist.Gamma(shape, rate)
        
        
        # If it is not censored, our observations are the directly observed
        # values of y
        with pyro.poutine.mask(mask = (censored_label == 0.0)):
            observation = pyro.sample("obs", outcome_dist, obs=y)
            
        # If it is censored, our observations are the observed
        # censoring lables, and those come from
        # a Bernoulli distribution where the probability
        # of censoring is defined by the CDF of the Gamma distribution
        with pyro.poutine.mask(mask = (censored_label == 1.0)):

            truncation_prob = torch.tensor(1 - scipygamma(shape.detach(), rate.detach()).cdf(y).astype(np.float32))
            #print(truncation_prob.min(), truncation_prob.max())
            
            censored_observation = pyro.sample("censorship",
                                               dist.Bernoulli(truncation_prob),
                                               obs=torch.tensor(1.0))
        

Notice since `pyro` doesn't yet implement a CDF method for its Gamma distribution I had to use the Gamma distribution from `scipy`.

Now we can run our model again, but this time sending both the observed prices and observed censoring of data.

In [ ]:
from sklearn.preprocessing import StandardScaler



X_train_scaled = my_x_scaler.fit_transform(X_train)
X_test_scaled = my_x_scaler.transform(X_test)

y_max = y_train.max()
y_train_scaled = y_train/y_max
y_test_scaled = y_test/y_max

X_train_torch = torch.tensor(X_train_scaled, dtype=torch.float64)
y_train_torch = torch.tensor(y_train_scaled, dtype=torch.float64)

# Find censored labels
numpy_censored_label = np.where(y_train >= y_max, 1.0, 0.0)
censored_label = torch.tensor(numpy_censored_label, dtype=torch.float64)





pyro.clear_param_store()
my_kernel = NUTS(model_gamma_cen, max_tree_depth=7)


my_mcmc4 = MCMC(my_kernel,
                num_samples=SAMPLE_NUMBER,
                warmup_steps=100)


start_time = time.time()
my_mcmc4.run(X_train_torch,
             y_train_torch, 
             california.feature_names, 
             censored_label)
end_time = time.time()

print(f'Inference ran for {round(end_time -  start_time, 2)} seconds')

super_results.loc['runtime', 'MCMC gamma+cen'] = round(end_time -  start_time, 2)



my_mcmc4.summary()

In [ ]:
beta4_df = pd.DataFrame(my_mcmc4.get_samples())
beta4_df = create_beta_df(beta4_df, my_x_scaler, california.feature_names)

beta4_df['beta_intercept'] += np.log(y_max)

In [ ]:
linear_combination = predict_linear_combination(beta4_df, X_test)

In [ ]:
y_pred = np.exp(linear_combination)
y_pred_small = y_pred[y_test<max_threshold]
y_pred= np.where(y_pred > max_threshold, max_threshold, y_pred)

In [ ]:
plt.figure(figsize=(15, 5))
plt.scatter(y_pred, y_test, c="blue", edgecolor="black", label=f'R^2={round(r2_score(y_test, y_pred), 2)}')
plt.title("Predicted House Price vs Actual House Price", size=22)
plt.xlabel("Predicted House Price (in $100K)", size=16)
plt.ylabel("True House Price (in $100K)", size=16)
plt.ticklabel_format(style='plain')
plt.legend()
plt.show()

In [ ]:
super_results.loc['R^2 censored', 'MCMC gamma+cen'] = r2_score(y_test, y_pred)
print(r2_score(y_test, y_pred))

In [ ]:
plt.figure(figsize=(15, 5))
plt.scatter(y_pred_small, y_small, c="blue", edgecolor="black", label=f'R^2={round(r2_score(y_small, y_pred_small),2)}')
plt.title("Predicted House Price vs Actual House Price", size=22)
plt.xlabel("Predicted House Price (in $100K)", size=16)
plt.ylabel("True House Price (in $100K)", size=16)
plt.ticklabel_format(style='plain')
plt.legend()
plt.show()

In [ ]:
super_results.loc['R^2 small', 'MCMC gamma+cen'] = r2_score(y_small, y_pred_small)
print(r2_score(y_small, y_pred_small))

In [ ]:
draw_coefficients(beta4_df)

## SVI

In [ ]:
from torch.distributions import constraints
def guide_gamma_cen(X, y, column_names, censored_label):
    pyro.enable_validation(True) 
    
    
    mu_intercept = pyro.param("mu_intercept", 
                              torch.tensor(0.0, dtype=torch.float64))
    sigma_intercept = pyro.param("sigma_intercept", 
                                 torch.tensor(1.0, dtype=torch.float64), 
                                 constraint=constraints.positive)

    # Again find the linear combination
    linear_combination = pyro.sample(f"beta_intercept", 
                                     dist.Normal(mu_intercept, sigma_intercept))
    
    
    mu_coef = pyro.param(f"mu_betas", 
                         torch.zeros(X.shape[1], dtype=torch.float64))
    sigma_coef = pyro.param(f"sigma_betas", 
                            torch.ones(X.shape[1], dtype=torch.float64), 
                            constraint=constraints.positive)
        
    betas = pyro.sample(f"beta_coefficients", dist.Normal(mu_coef, sigma_coef).expand([X.shape[1]]).to_event(1)).double()
    

    # Let's try and fit a Gamma distribution on the rate parameter
    
    shape_r = pyro.param("shape_r", torch.tensor(10.0, dtype=torch.float64), constraint=constraints.positive)
    rate_r = pyro.param("rate_r", torch.tensor(10.0, dtype=torch.float64), constraint=constraints.positive)

    
    rate = pyro.sample("rate", dist.Gamma(shape_r, rate_r))
    
        

In [ ]:
from pyro.infer import SVI, Trace_ELBO, TraceGraph_ELBO
from pyro.infer import Predictive
from pyro.optim import ClippedAdam
from pyro.infer.autoguide import AutoDiagonalNormal, AutoMultivariateNormal

# Scale the data
X_train_scaled = my_x_scaler.fit_transform(X_train)
X_test_scaled = my_x_scaler.transform(X_test)

y_max = y_train.max()
y_train_scaled = y_train/y_max
y_test_scaled = y_test/y_max

X_train_torch = torch.tensor(X_train_scaled, dtype=torch.float64)
y_train_torch = torch.tensor(y_train_scaled, dtype=torch.float64)

# Find censored labels
numpy_censored_label = np.where(y_train >= y_max, 1.0, 0.0)
censored_label = torch.tensor(numpy_censored_label, dtype=torch.float64)





pyro.clear_param_store()
my_guide = guide_gamma_cen
#my_guide = AutoDiagonalNormal(model_gamma_cen)



my_svi = SVI(model=model_gamma_cen,
             guide= my_guide,
             optim=ClippedAdam({"lr": 0.01, 'clip_norm': 1.0}),
             loss=TraceGraph_ELBO())

losses = []

start_time = time.time()
for i in range(10000):

    loss = my_svi.step(X_train_torch,
                       y_train_torch, 
                       california.feature_names, 
                       censored_label)
    
    normalized_loss = loss/X_train_torch.shape[0]
    losses.append(normalized_loss)
    if (i % 1000 == 0):
        print(f'iter: {i}, normalized loss:{round(normalized_loss,2)}')
        
        

predictive = Predictive(model=model_gamma_cen,
                        guide= my_guide,
                        num_samples=SAMPLE_NUMBER) 

samples = predictive(X_train_torch,
                     y_train_torch, 
                     california.feature_names, 
                     censored_label)

        
end_time = time.time()

print(f'Inference ran for {round(end_time -  start_time, 2)} seconds')

super_results.loc['runtime', 'SVI gamma+cen'] = round(end_time -  start_time, 2)





In [ ]:
beta_svi_df = {}
for key, values in samples.items():
    if ("beta_" in key) or ("rate" in key):
        if ("beta_coefficients" in key):
            values = values.view(values.shape[0], values.shape[-1])
        else:
            values = values.view(values.shape[0], )
        
        beta_svi_df[key] = values.detach()
        
beta_svi_df = pd.DataFrame(beta_svi_df)

losses = np.array(losses)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(np.log(losses+1))
plt.show()

In [ ]:
beta_svi_df = create_beta_df(beta_svi_df, my_x_scaler, california.feature_names)
beta_svi_df['beta_intercept'] += np.log(y_max)

In [ ]:
linear_combination = predict_linear_combination(beta4_df, X_test)

In [ ]:
y_pred = np.exp(linear_combination)
y_pred_small = y_pred[y_test<max_threshold]
y_pred= np.where(y_pred > max_threshold, max_threshold, y_pred)

In [ ]:
plt.figure(figsize=(15, 5))
plt.scatter(y_pred, y_test, c="blue", edgecolor="black", label=f'R^2={round(r2_score(y_test, y_pred), 2)}')
plt.title("Predicted House Price vs Actual House Price", size=22)
plt.xlabel("Predicted House Price (in $100K)", size=16)
plt.ylabel("True House Price (in $100K)", size=16)
plt.ticklabel_format(style='plain')
plt.legend()
plt.show()

In [ ]:
super_results.loc['R^2 censored', 'SVI gamma+cen'] = r2_score(y_test, y_pred)
print(r2_score(y_test, y_pred))

In [ ]:
plt.figure(figsize=(15, 5))
plt.scatter(y_pred_small, y_small, c="blue", edgecolor="black", label=f'R^2={round(r2_score(y_small, y_pred_small),2)}')
plt.title("Predicted House Price vs Actual House Price", size=22)
plt.xlabel("Predicted House Price (in $100K)", size=16)
plt.ylabel("True House Price (in $100K)", size=16)
plt.legend()
plt.ticklabel_format(style='plain')
plt.show()

In [ ]:
super_results.loc['R^2 small', 'SVI gamma+cen'] = r2_score(y_small, y_pred_small)
print(r2_score(y_small, y_pred_small))

In [ ]:
draw_coefficients(beta_svi_df)

In [ ]:
super_end = time.time()

print(f'Notebook runtime: {round((super_end -  super_start)/60, 2)} minutes')


In [ ]:
super_results